In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arxiv-dataset-trimmed/test.json
/kaggle/input/arxiv/arxiv-metadata-oai-snapshot.json


In [2]:
# import some libraries for NLP and visualize

%matplotlib inline

import matplotlib.pyplot as plt
import spacy

In [3]:
# Using readlines() 
# test.json is a reduced data set of ../input/arxiv/arxiv-metadata-oai-snapshot.json
file = open('/kaggle/input/arxiv-dataset-trimmed/test.json', 'r') 
Lines = file.readlines() 

In [4]:
len(Lines)

17783

In [6]:
import json 
import time
if __name__ == '__main__':
    start = time.time()
    all_ = []
    for count in range(0,10000):
        current_json = json.loads(Lines[count])
        current_ = [current_json['id'], current_json['categories'], current_json['abstract']]
        all_.append(current_)
    elapsed_time = time.time() - start
    print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:0.21630311012268066[sec]


In [8]:
all_df = pd.DataFrame(all_, columns=["id", "categories", "abstract"])
all_df.head(10)

,id,categories,abstract
0,0704.0100,gr-qc,The topological structure of the event horiz...
1,0704.0200,hep-ph nucl-ex,The electromagnetic polarizabilities of the ...
2,0704.0300,cond-mat.mes-hall cond-mat.mtrl-sci,We present an experimental investigation on ...
3,0704.0400,nlin.SI cond-mat.stat-mech hep-th,We review the algebraic construction of the ...
4,0704.0500,math.GR,We prove that if a group is nilpotent (resp....
5,0704.0600,math.CO math.DS,"We introduce a new class of ""random"" subsets..."
6,0704.0700,physics.chem-ph,This work reports a theoretical study of the...
7,0704.0800,quant-ph,We present a quantum auction protocol using ...
8,0704.0900,cond-mat.supr-con cond-mat.mes-hall,We compute the current voltage characteristi...
9,0704.1000,hep-ex,We report a measurement of D0-D0bar mixing i...


Make 'categories' to more broader 'categories_broad' and encoded it into numerical labels.

In [9]:
import re

all_df['categories_broad'] = all_df['categories']

for ind in all_df.index:
    pos_period = re.search('\.',all_df['categories'][ind].split()[0])
    if pos_period != None:
        all_df['categories_broad'][ind] = re.match('(.*)\.',all_df['categories'][ind].split()[0])[1]
    else:
        all_df['categories_broad'][ind] = all_df['categories'][ind].split()[0]

In [10]:
all_df['categories_broad'].describe()

count     10000
unique       19
top        math
freq       2364
Name: categories_broad, dtype: object

In [11]:
# Encode categories_broad to number
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
all_df['categories_broad_enc'] = le.fit_transform(all_df['categories_broad'])

Using Spacy, let's predict the category from the abstract.

In [12]:
# Load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

In [ ]:
abstracts = all_df[['abstract']]
with nlp.disable_pipes():
    vectors = np.array([nlp(abstract.abstract).vector for idx, abstract in abstracts.iterrows()])
    
vectors.shape

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectors, all_df['categories_broad_enc'], 
                                                    test_size=0.2, random_state=1)

# Create the LinearSVC model
model = LinearSVC(random_state=1, dual=False)
# Fit the model
model.fit(X_train, y_train)

# Uncomment and run to see model accuracy
print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')

In [ ]:
# Scratch space in case you want to experiment with other models
from sklearn.ensemble import RandomForestClassifier

second_model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
second_model.fit(X_train, y_train)
print(f'Model test accuracy: {second_model.score(X_test, y_test)*100:.3f}%')

In [ ]:
# df_t.to_csv('out.csv')  